In [4]:
require(Seurat)
require(dplyr)
library(ggplot2)

In [2]:
Hsap <- readRDS('../2.SAM_reclustering/SAM/Hsap_Ast.15.30.2500.DonorID.rds')
Mmus <- readRDS('../2.SAM_reclustering/SAM/Mmus_Ast.15.30.2500.DonorID.rds')
Pvit <- readRDS('../2.SAM_reclustering/SAM/Pvit_Ast.15.30.2500.DonorID.rds')
Pmar <- readRDS('../2.SAM_reclustering/SAM/Pmar_Ast.15.30.2500.DonorID.rds')

In [3]:
Hsap <- FindNeighbors(Hsap, dims = 1:25)
Hsap <- FindClusters(Hsap, resolution = 0.3, cluster.name = 'AST.res.0.3')

Mmus <- FindNeighbors(Mmus, dims = 1:25)
Mmus <- FindClusters(Mmus, resolution = 0.3, cluster.name = 'AST.res.0.3')

Pvit <- FindNeighbors(Pvit, dims = 1:25)
Pvit <- FindClusters(Pvit, resolution = 0.3, cluster.name = 'AST.res.0.3')

Pmar <- FindNeighbors(Pmar, dims = 1:25)
Pmar <- FindClusters(Pmar, resolution = 0.3, cluster.name = 'AST.res.0.3')

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 18895
Number of edges: 663916

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9159
Number of communities: 10
Elapsed time: 16 seconds


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 19015
Number of edges: 679420

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9458
Number of communities: 15
Elapsed time: 17 seconds


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 31316
Number of edges: 1029042

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8995
Number of communities: 10
Elapsed time: 37 seconds


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 6389
Number of edges: 223476

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9225
Number of communities: 10
Elapsed time: 4 seconds


In [34]:
# fast annotation for mouse
res = c()
for (i in unique(Mmus@meta.data$`AST.res.0.3`)){
    celli <- colnames(Mmus)[which(Mmus@meta.data$`AST.res.0.3` == i)]
    for (j in unique(Mmus@meta.data$Description)){
        cellj <- colnames(Mmus)[which(Mmus@meta.data$Description == j)]
        inter <- length(intersect(celli, cellj))/length(celli)
        res <- rbind(res, c(i, j, inter))
    }
}
res = as.data.frame(res)
res$V3 <- as.double(res$V3)
ops = res[res$V3 > 0.5,]
ops$V4 <- paste(ops$V2, ops$V1, sep = '_')

In [35]:
# fast annotation for mouse
Mmus@meta.data$AST <- vapply(Mmus@meta.data$`AST.res.0.3`, FUN = function(x){
    if (x %in% ops$V1){
        return(ops[ops$V1 == x, 'V2'])
    } else {
        return("Non-telencephalon astrocytes")
    }
}, FUN.VALUE = character(1))

In [21]:
# fast annotation for human based on location
Hsap@meta.data$Tissue <- vapply(as.character(Hsap@meta.data$Tissue), FUN = function(x){
                                      strsplit(x, split = "-")[[1]][1]
},   FUN.VALUE = character(1))

res = c()
for (i in unique(Hsap@meta.data$`AST.res.0.3`)){
    celli <- colnames(Hsap)[which(Hsap@meta.data$`AST.res.0.3` == i)]
    for (j in unique(Hsap@meta.data$Tissue)){
        cellj <- colnames(Hsap)[which(Hsap@meta.data$Tissue == j)]
        inter <- length(intersect(celli, cellj))/length(celli)
        res <- rbind(res, c(i, j, inter))
    }
}
res = as.data.frame(res)
res$V3 <- as.double(res$V3)
res <- res %>% group_by(V1) %>% filter(V3 == max(V3))
res$V4 <- paste(res$V2, res$V1, sep = '_')

Hsap@meta.data$AST <- vapply(Hsap@meta.data$`AST.res.0.3`, FUN = function(x){
    return(unlist(res[res$V1 == x, 'V4']))
}, FUN.VALUE = character(1))

In [22]:
# fast annotation for lizard based on location
Pvit@meta.data$Region <- vapply(as.character(Pvit@meta.data$Region), FUN = function(x){
                                      strsplit(x, split = "-")[[1]][1]
},   FUN.VALUE = character(1))

res = c()
for (i in unique(Pvit@meta.data$`AST.res.0.3`)){
    celli <- colnames(Pvit)[which(Pvit@meta.data$`AST.res.0.3` == i)]
    for (j in unique(Pvit@meta.data$Region)){
        cellj <- colnames(Pvit)[which(Pvit@meta.data$Region == j)]
        inter <- length(intersect(celli, cellj))/length(celli)
        res <- rbind(res, c(i, j, inter))
    }
}
res = as.data.frame(res)
res$V3 <- as.double(res$V3)
res <- res %>% group_by(V1) %>% filter(V3 == max(V3))
res$V4 <- paste(res$V2, res$V1, sep = '_')

Pvit@meta.data$AST <- vapply(Pvit@meta.data$`AST.res.0.3`, FUN = function(x){
    return(unlist(res[res$V1 == x, 'V4']))
}, FUN.VALUE = character(1))

In [23]:
# fast annotation for lamprey based on location
Pmar@meta.data$Putative_location <- vapply(as.character(Pmar@meta.data$Putative_location), FUN = function(x){
                                      strsplit(x, split = "-")[[1]][1]
},   FUN.VALUE = character(1))

res = c()
for (i in unique(Pmar@meta.data$`AST.res.0.3`)){
    celli <- colnames(Pmar)[which(Pmar@meta.data$`AST.res.0.3` == i)]
    for (j in unique(Pmar@meta.data$Putative_location)){
        cellj <- colnames(Pmar)[which(Pmar@meta.data$Putative_location == j)]
        inter <- length(intersect(celli, cellj))/length(celli)
        res <- rbind(res, c(i, j, inter))
    }
}
res = as.data.frame(res)
res$V3 <- as.double(res$V3)
res <- res %>% group_by(V1) %>% filter(V3 == max(V3))
res$V4 <- paste(res$V2, res$V1, sep = '_')

Pmar@meta.data$AST <- vapply(Pmar@meta.data$`AST.res.0.3`, FUN = function(x){
    return(unlist(res[res$V1 == x, 'V4']))
}, FUN.VALUE = character(1))

In [1]:
Hsap <- readRDS('../2.SAM_reclustering/SAM/Hsap_Ast.15.30.2500.DonorID.rds')
Mmus <- readRDS('../2.SAM_reclustering/SAM/Mmus_Ast.15.30.2500.DonorID.rds')
Pvit <- readRDS('../2.SAM_reclustering/SAM/Pvit_Ast.15.30.2500.DonorID.rds')
Pmar <- readRDS('../2.SAM_reclustering/SAM/Pmar_Ast.15.30.2500.DonorID.rds')

In [ ]:
Mmus@meta.data$AST <- factor(Mmus@meta.data$AST, levels = rev(c('Olfactory astrocytes', 'Subventricular zone radial glia-like cells',
                                                           'Telencephalon astrocytes, protoplasmic', 'Telencephalon astrocytes, fibrous',
                                                           'Non-telencephalon astrocytes', 'Non-telencephalon astrocytes, protoplasmic',
                                                           'Non-telencephalon astrocytes, fibrous', 'Dorsal midbrain Myoc-expressing astrocyte-like',
                                                           'Bergmann glia')))

Hsap@meta.data$AST <- factor(Hsap@meta.data$AST, levels = rev(c('Cerebral cortex (Cx) _0', 'Cerebral cortex (Cx) _4',
                                                               'Basal forebrain (BF) _9', 'Basal nuclei (BN) _2', 'Basal nuclei (BN) _8',
                                                               'Head of hippocampus (HiH) _1', 'Thalamus (THM) _5', 'Hypothalamus (HTH) _7',
                                                               'Cerebellum (CB) _3', 'Cerebellum (CB) _6')))

Pvit@meta.data$AST <- factor(Pvit@meta.data$AST, levels = rev(c('Telencephalon_0', 'Telencephalon_2', 'Telencephalon_7',
                                                               'Thalamus+Hypothalamus+Midbrain_1', 'Thalamus+Hypothalamus+Midbrain_3',
                                                               'Thalamus+Hypothalamus+Midbrain_4','Thalamus+Hypothalamus+Midbrain_6',
                                                               'Thalamus+Hypothalamus+Midbrain_8','Thalamus+Hypothalamus+Midbrain_9',
                                                               'Cerebellum_5')))

Pmar@meta.data$AST <- factor(Pmar@meta.data$AST, levels = rev(c('Prosencephalon_1','Prosencephalon_3','Prosencephalon_7',
                                                               'Prosencephalon_8','Diencephalon_4','Diencephalon/Mesencephalon_5',
                                                               'Diencephalon/Mesencephalon_6','Diencephalon/Mesencephalon_9',
                                                               'Mesencephalon/Rhombencephalon_0','Mesencephalon/Rhombencephalon_2')))

Idents(Mmus) <- 'AST'
Idents(Hsap) <- 'AST'
Idents(Pvit) <- 'AST'
Idents(Pmar) <- 'AST'

In [46]:
p2 <- DotPlot(Mmus, scale.max = 10, features = c('Sphk1','Foxg1','Slc17a7','Slc17a6','Igsf1','En1','Aplnr','Gdf10','Pax3', 'Mybpc1',
                                   'Tfap2c','Gfap')) + theme(axis.text.x = element_text(angle = 90))
p1 <- DotPlot(Hsap, scale.max = 10, features = c('ENSG00000176170','ENSG00000176165','ENSG00000104888','ENSG00000091664',
                                 'ENSG00000147255','ENSG00000163064','ENSG00000134817','ENSG00000266524','ENSG00000135903', 'ENSG00000196091',
                                   'ENSG00000087510','ENSG00000131095')) + theme(axis.text.x = element_text(angle = 90))
p3 <- DotPlot(Pvit, scale.max = 10, features = toupper(c('Sphk1','Foxg1','Slc17a7','Slc17a6','Igsf1','En1','Aplnr','DRAXIN','Pax3', 'Mybpc1',
                                   'Tfap2c','Gfap'))) + theme(axis.text.x = element_text(angle = 90))
p4 <- DotPlot(Pmar, scale.max = 10, features = c('MSTRG.16654', 'MSTRG.10939','PMZ-0043056','MSTRG.857',
                                        'MSTRG.4830', 'MSTRG.18473')) + theme(axis.text.x = element_text(angle = 90))
pdf('figures/AST_subtype.dotplot.pdf', width = 32, height = 8)
p1|p2|p3|p4
dev.off()

Warning message:
“The following requested variables were not found: IGSF1”


pdf 
  2

In [51]:
saveRDS(Hsap, '../2.SAM_reclustering/SAM/Hsap_Ast.15.30.2500.DonorID.rds')
saveRDS(Mmus, '../2.SAM_reclustering/SAM/Mmus_Ast.15.30.2500.DonorID.rds')
saveRDS(Pvit, '../2.SAM_reclustering/SAM/Pvit_Ast.15.30.2500.DonorID.rds')
saveRDS(Pmar, '../2.SAM_reclustering/SAM/Pmar_Ast.15.30.2500.DonorID.rds')

In [ ]:
Hsap <- readRDS('../2.SAM_reclustering/SAM/Hsap_Ast.15.30.2500.DonorID.rds')
Mmus <- readRDS('../2.SAM_reclustering/SAM/Mmus_Ast.15.30.2500.DonorID.rds')
Pvit <- readRDS('../2.SAM_reclustering/SAM/Pvit_Ast.15.30.2500.DonorID.rds')
Pmar <- readRDS('../2.SAM_reclustering/SAM/Pmar_Ast.15.30.2500.DonorID.rds')